In [224]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from proj1_helpers import load_csv_data
from functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [225]:
train_set = load_csv_data('Data/train.csv', sub_sample=True)

In [226]:
train_set_pd = pd.read_csv('Data/train.csv')
train_set_pd.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [227]:
x_train = train_set[1]
y_train = train_set[0]

In [228]:
def to_zero_for_neg(x):
    if x < 0:
        return 0
    else:
        return 1

In [229]:
for i in range (len(y_train)):
    if(y_train[i] < 0):
        y_train[i] = 0
y_train

array([ 1.,  0.,  0., ...,  1.,  0.,  1.])

In [230]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient_least_square(y, tx, w)
        loss = compute_mse(y, tx, w)
        w = w - gamma*gradient
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return loss, w

In [231]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):    
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_mse(minibatch_y, minibatch_tx, w)
            new_w = w - gamma*gradient
            w = new_w
            ws.append(w)
            losses.append(loss)
            print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
            bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [232]:
def least_squares(y, tx):
    A = tx.T@tx
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [287]:
def ridge_regression(y, tx, lambda_):
    X = tx.T@tx
    N = X.shape[0]
    A = (X + 2*N*lambda_*np.identity(N))
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [296]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    losses, ws = gradient_descent_log_reg(y, tx, initial_w, max_iters, gamma)
    w = ws[len(ws)-1]
    loss = losses[len(losses)-1]
    return w, loss

In [297]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    return w, loss

**TESTS**

In [299]:
### WRONG RESULTS
logistic_regression(y_train, x_train, np.zeros(x_train.shape[1]), 50, 0.7)

Gradient Descent(0/49): loss=3465.6359037995257, w0=202309.98480000003, w1=-50705.06139999999
Gradient Descent(1/49): loss=19675.556709798744, w0=187195.62540000005, w1=-48799.00549999999
Gradient Descent(2/49): loss=37198.24448740738, w0=546965.8012000001, w1=-146435.64459999994
Gradient Descent(3/49): loss=19698.582580728595, w0=536047.2418000001, w1=-144904.04249999995
Gradient Descent(4/49): loss=19445.298000500312, w0=516285.4956000001, w1=-141131.09569999995
Gradient Descent(5/49): loss=17568.689519718006, w0=580048.3941, w1=-108250.76219999995
Gradient Descent(6/49): loss=37601.19722867963, w0=930822.0292, w1=-207628.5567999999
Gradient Descent(7/49): loss=19617.99203247414, w0=910812.5697999999, w1=-205044.7762999999
Gradient Descent(8/49): loss=19525.888548754774, w0=885044.9279, w1=-201581.4275999999
Gradient Descent(9/49): loss=19445.29800050038, w0=954625.9569, w1=-167075.81099999993
Gradient Descent(10/49): loss=37486.06787403042, w0=1276886.8042, w1=-273884.8776999999
Gra

(array([  1.78808379e+06,  -7.92194770e+05,  -4.75110404e+05,
          2.28788819e+05,  -3.07284797e+05,  -6.80176865e+04,
         -3.16077281e+05,  -7.83606985e+03,  -9.70922802e+04,
         -3.36689164e+05,  -1.67868572e+04,   1.29444816e+04,
         -3.04459065e+05,   1.97890835e+05,   2.11908795e+03,
         -1.84352980e+03,  -2.47809878e+05,   3.46447010e+03,
          2.73551460e+03,  -7.13471199e+04,   6.76448500e+02,
         -5.40102068e+05,  -8.98590000e+03,   3.99421799e+05,
          3.63000059e+05,   3.64647529e+05,  -4.95357899e+05,
         -3.03309165e+05,  -3.05644151e+05,  -2.86769351e+05]),
 17062.1203592614)